In [2]:
import tensorflow as tf 
import tensorflow.keras as keras

In [3]:
cell = keras.layers.SimpleRNNCell(3)  # 这里的3代表的这个单元的输出的在维度上是3(前向传递3次)
cell.build(input_shape = (None,4)) # 4是输入的的feature宽度

In [5]:
cell.trainable_variables
# kernel:0 是Wxh, 是与输入进行计算的矩阵
# recurrent_kernel:0 是Whh  是与原状态进行计算的矩阵

[<tf.Variable 'kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.35893172,  0.34640408, -0.7190231 ],
        [-0.38354695,  0.16251683,  0.65487933],
        [ 0.36017287, -0.6169188 , -0.762123  ],
        [ 0.32048357, -0.4431951 ,  0.33596396]], dtype=float32)>,
 <tf.Variable 'recurrent_kernel:0' shape=(3, 3) dtype=float32, numpy=
 array([[-0.9002893 ,  0.42969328,  0.06959017],
        [ 0.37536383,  0.84730864, -0.37572592],
        [-0.22041129, -0.3121404 , -0.9241143 ]], dtype=float32)>,
 <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [13]:
# 模拟一个简单的RNN
simp_cell = keras.layers.SimpleRNNCell(64)
simp_cell.build(input_shape = (None,100))
input_data = tf.random.normal([8,80,100])
xt0 = input_data[:,0,:]

out,xt1 = simp_cell(xt0,[tf.zeros([8,64])])

In [14]:
out.shape,xt1[0].shape

(TensorShape([8, 64]), TensorShape([8, 64]))

In [15]:
simp_cell.trainable_variables

[<tf.Variable 'kernel:0' shape=(100, 64) dtype=float32, numpy=
 array([[-0.00640079, -0.12984654, -0.02969642, ...,  0.06039654,
         -0.08218826,  0.01885659],
        [ 0.10211475, -0.07486852, -0.06810126, ..., -0.0979297 ,
         -0.05193917, -0.10632212],
        [ 0.03461631, -0.08166032,  0.02634825, ...,  0.08819743,
         -0.05791144, -0.14014924],
        ...,
        [ 0.1257103 , -0.1828993 , -0.1486468 , ...,  0.0077257 ,
          0.03304191,  0.05503471],
        [ 0.1763599 ,  0.0296919 , -0.17342275, ..., -0.13892391,
          0.14954974,  0.07974298],
        [-0.1318248 ,  0.01640885,  0.13628323, ..., -0.06069523,
          0.17974795, -0.03248492]], dtype=float32)>,
 <tf.Variable 'recurrent_kernel:0' shape=(64, 64) dtype=float32, numpy=
 array([[-0.05354011,  0.18444194, -0.12552366, ..., -0.01768488,
         -0.16753767, -0.19983152],
        [ 0.03977003,  0.3143643 ,  0.2694412 , ...,  0.09904367,
         -0.07608538,  0.04141185],
        [-0.204587

In [18]:
# 多层RNN的实现
simp_rnn_cell0 = keras.layers.SimpleRNNCell(64)
simp_rnn_cell1 = keras.layers.SimpleRNNCell(64)

input_data = tf.random.normal([8,80,100])
xt0 = input_data[:,0,:]

state0 = [tf.zeros([8,64])]
state1 = [tf.zeros([8,64])]

out0,state0 = simp_rnn_cell0(xt0,state0)
out1,state1 = simp_rnn_cell1(out0,state1)

In [19]:
# 对批量的句子进行迭代
for words in tf.unstack(input_data,axis=1): # 在1维度上进行迭代
    out0,state0 = simp_rnn_cell0(words,state0)
    out1,state1 = simp_rnn_cell1(out0,state1)


In [28]:
units = 64 # 这个值代表的是RNN网络cell的输出维度
# 一种比较简单的RNN网购狗仔方法:使用Sequential对象进行管理
rnn = keras.Sequential([
    keras.layers.SimpleRNN(units,dropout = 0.5, return_sequences = True, unroll = True), 
    # return_sequences 表示每次时间节点的计算都需要范湖一个值给下一层网络当做输入使用, 如果不设置这个值, 那么网络层只返回最后一个时间节点计算得到的值值
    keras.layers.SimpleRNN(units,dropout = 0.5, unroll = True)
])
# SimpleRNN相对于SimpleRNNCell的区别就是, SimpleRNN不需要我们对输入项进行时间维度的抽取切片, 系统会自动完成
input_data = tf.random.normal([8,80,100])
out = rnn(input_data)
# out是最后一层的最优一个时间节点的计算输出

In [29]:
out.shape

TensorShape([8, 64])